In [1]:
import torch
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image

import matplotlib.pyplot as plt

In [3]:
def display_images():
    fig, subplots = plt.subplots(1,5, figsize=(13,6))
    for i in range(5):
        axil = subplots.flat[i]
        original_img = Image.fromarray(cifar10.data[i])
        axil.imshow(original_img)
        axil.set_title('original_img')

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [6]:
# 데이터셋의 루트 디렉터리와 이미지 폴더를 지정합니다.
root_dir = '../data/'  # 데이터셋의 루트 디렉터리 경로
train_folder = 'kfood_train/train'  # 이미지가 있는 폴더의 이름
val_folder = 'kfood_val/val'  # 이미지가 있는 폴더의 이름

In [8]:
# 이미지 전처리를 위한 변환(transform)을 정의합니다.
transform = transforms.Compose([
    transforms.Resize((244, 244)),  # 이미지 크기를 조절합니다.
    transforms.ToTensor(),  # 이미지를 Tensor로 변환합니다.
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지를 정규화합니다.
])

In [14]:
# 데이터셋을 불러옵니다.
trainset = torchvision.datasets.ImageFolder(
    root=root_dir + train_folder,
    transform=transform
)

In [15]:
# 데이터셋을 불러옵니다.
valset = torchvision.datasets.ImageFolder(
    root=root_dir + val_folder,
    transform=transform
)

In [59]:
# 데이터 로더를 설정합니다.
batch_size = 8 # 배치 크기를 지정합니다.
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,  # 데이터를 섞어서 로드합니다.
    num_workers=4  # 데이터를 병렬로 로드할 워커 수를 지정합니다.
)

In [60]:
# 데이터 로더를 설정합니다.
batch_size = 8  # 배치 크기를 지정합니다.
valloader = torch.utils.data.DataLoader(
    valset,
    batch_size=batch_size,
    shuffle=False,  # 데이터를 섞어서 로드합니다.
    num_workers=4  # 데이터를 병렬로 로드할 워커 수를 지정합니다.
)

In [61]:
# 클래스 레이블을 확인합니다.
classes = trainset.classes
print(classes)

['갈비구이', '갈치구이', '감자전', '경단', '계란국', '계란말이', '계란후라이', '고등어구이', '곱창구이', '김밥', '김치볶음밥', '김치전', '꿀떡', '누룽지', '닭갈비', '더덕구이', '동그랑땡', '떡갈비', '떡국_만두국', '무국', '미역국', '북엇국', '불고기', '비빔밥', '삼겹살', '새우볶음밥', '생선전', '송편', '시래기국', '알밥', '유부초밥', '육개장', '잡곡밥', '장어구이', '조개구이', '조기구이', '주먹밥', '콩나물국', '파전', '호박전', '황태구이', '훈제오리']


In [62]:
# 클래스 레이블을 확인합니다.
classes = valset.classes
print(classes)

['갈비구이', '갈치구이', '감자전', '경단', '계란국', '계란말이', '계란후라이', '고등어구이', '곱창구이', '김밥', '김치볶음밥', '김치전', '꿀떡', '누룽지', '닭갈비', '더덕구이', '동그랑땡', '떡갈비', '떡국_만두국', '무국', '미역국', '북엇국', '불고기', '비빔밥', '삼겹살', '새우볶음밥', '생선전', '송편', '시래기국', '알밥', '유부초밥', '육개장', '잡곡밥', '장어구이', '조개구이', '조기구이', '주먹밥', '콩나물국', '파전', '호박전', '황태구이', '훈제오리']


In [63]:
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

In [64]:
resnet_pt = models.resnet18(pretrained=False)
# freezing
# for param in resnet_pt.parameters():
#     param.requires_grad = False

# fc layer 수정
fc_in_features = resnet_pt.fc.in_features
resnet_pt.fc = nn.Linear(fc_in_features, len(classes))
resnet_pt = resnet_pt.to(device)

In [65]:
from torchsummary import summary
summary(resnet_pt, (3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,

In [66]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_pt.parameters() , lr = 0.001 , momentum = 0.9)

In [67]:
resnet_pt.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [68]:
for epoch in range(3):
    running_loss = 0.0
    running_val_loss = 0.0  # validation loss를 저장하기 위한 변수 추가

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet_pt(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # validation loss 계산
        if i % 100 == 99:
            resnet_pt.eval()  # 모델을 평가 모드로 설정

            with torch.no_grad():  # 그라디언트 계산 비활성화
                for val_data in valloader:
                    val_inputs, val_labels = val_data[0].to(device), val_data[1].to(device)
                    val_outputs = resnet_pt(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()

            resnet_pt.train()  # 모델을 다시 훈련 모드로 설정

            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}, val_loss: {running_val_loss / len(valloader):.3f}')
            running_loss = 0.0
            running_val_loss = 0.0

print('finished Training')

[1,   100] loss: 0.190, val_loss: 3.793
[1,   200] loss: 0.184, val_loss: 3.531
[1,   300] loss: 0.175, val_loss: 3.483
[1,   400] loss: 0.175, val_loss: 3.514
[1,   500] loss: 0.172, val_loss: 3.326
[1,   600] loss: 0.171, val_loss: 3.328
[1,   700] loss: 0.172, val_loss: 3.311
[1,   800] loss: 0.167, val_loss: 3.373
[1,   900] loss: 0.163, val_loss: 3.246
[1,  1000] loss: 0.166, val_loss: 3.264
[1,  1100] loss: 0.161, val_loss: 3.074
[1,  1200] loss: 0.163, val_loss: 3.150
[1,  1300] loss: 0.159, val_loss: 3.108
[1,  1400] loss: 0.161, val_loss: 3.036
[1,  1500] loss: 0.156, val_loss: 3.100
[1,  1600] loss: 0.159, val_loss: 3.116


KeyboardInterrupt: 

In [56]:

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs

with torch.no_grad():
    for data in valloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = resnet_pt(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _ , predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy {100 * correct // total} %') 

Accuracy 8 %


In [41]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs

with torch.no_grad():
    for data in trainloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = resnet_pt(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _ , predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy {100 * correct // total} %')

Accuracy 80 %
